In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error


In [ ]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore")

In [ ]:
### Read the data

In [ ]:
train = pd.read_csv('C:\\Prasanna\\Anagha\\Regularization\\train.csv')
X_test = pd.read_csv('C:\\Prasanna\\Anagha\\Regularization\\test.csv')

In [ ]:
## split into Training and test

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop('SalePrice', axis=1)
y = train.SalePrice

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=1121218)

In [ ]:
## Numerical and categorical feature seperation

In [ ]:
numerical_features = X_train.select_dtypes(include='number').columns.tolist()
print(f'There are {len(numerical_features)} numerical features:', '\n')
print(numerical_features)

categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()
print(f'There are {len(categorical_features)} categorical features:', '\n')
print(categorical_features)

There are 37 numerical features: 

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
There are 43 categorical features: 

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Func

In [ ]:
## Using sklearn pipeline , compute missing value with mean, scale using minmax and convert categorical into OHE

In [ ]:
numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
## Combine numeric and categorical using column transformer

In [ ]:
from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('category', categorical_pipeline, categorical_features)
])

In [ ]:
## Create LinearRegression , Lasso, Ridge and elasticnet pipelines

In [ ]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

ols_pipeline= Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', LinearRegression())
])


lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', Lasso())
])



Ridge_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', Ridge())
])



ElasticNet_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', ElasticNet())
])

In [ ]:
### Use grid search for finding repective optimum lambda
### elastic net==> l1_ratio    { 1==> lasso
###                              0==> Ridge}
###                     alpha==> penalty

In [ ]:
from sklearn.model_selection import GridSearchCV

param_dict = {'model__alpha': np.arange(1,2000,10)}

lasso_search = GridSearchCV(lasso_pipeline, param_dict,
                      cv=3,
                      scoring='neg_mean_absolute_error')
lasso_model = lasso_search.fit(X_train, y_train)


In [ ]:
print('Lasso Best score:', abs(lasso_search.best_score_))


Lasso Best score: 16477.296178859906


In [ ]:
Ridge_search = GridSearchCV(Ridge_pipeline, param_dict,
                      cv=3,
                      scoring='neg_mean_absolute_error')
Ridge_model = Ridge_search.fit(X_train, y_train)



In [ ]:
print('Ridge Best score:', abs(Ridge_search.best_score_))

Ridge Best score: 17751.500325108005


In [ ]:
param_dict = {'model__alpha': [1e-5,1.0, 10.0, 100.0,200,300,1000] ,
              'model__l1_ratio': np.arange(0, 1, 0.01)}
elastic_search = GridSearchCV(ElasticNet_pipeline, param_dict,
                      cv=3,
                      scoring='neg_mean_absolute_error')
elastic_model = elastic_search.fit(X_train, y_train)


In [ ]:
print('elastic Best score:', abs(elastic_model.best_score_))

elastic Best score: 17434.49415187335


In [ ]:
ols_model=ols_pipeline.fit(X_train, y_train)

In [ ]:
print(lasso_model.best_estimator_[1])
print(Ridge_model.best_estimator_[1])
print(elastic_model.best_estimator_[1])

AttributeError: 'Lasso' object has no attribute 'best_estimator_'

In [ ]:
### Choose beat parameters to rerun the regressions to get best fit

In [ ]:
lasso_model=Lasso(alpha=1000, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
ridge_model=Ridge(alpha=11, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
elastic_model=ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('lasso', lasso_model)
])


ridge_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('ridge', ridge_model)
])


elastic_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('elastic', elastic_model)
])


In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
ols=ols_pipeline.fit(X_train, y_train)
ols_pred=ols.predict(X_valid)
print("===ols====")
print("mse==>",mean_squared_error(y_valid,ols_pred))
print("mae==>",mean_absolute_error(y_valid,ols_pred))
print("R^2==>",ols.score(X_valid, y_valid))

===ols====
mse==> 3.665849810115942e+28
mae==> 22937764183473.92
R^2==> -5.164283426690424e+18


In [ ]:
lasso=lasso_pipeline.fit(X_train, y_train)
lasso_pred=lasso.predict(X_valid)
print("===lasso====")
print("mse==>",mean_squared_error(y_valid,lasso_pred))
print("mae==>",mean_absolute_error(y_valid,lasso_pred))
print("R^2==>",lasso.score(X_valid, y_valid))

===lasso====
mse==> 1604578591.933106
mae==> 21526.83618180848
R^2==> 0.773954224576366


In [ ]:
print("No: of Non zero coeficents",len([col for col in  lasso_pipeline.named_steps['lasso'].coef_ if abs(col) > 0 ]))

No: of Non zero coeficents 31


In [ ]:
ridge=ridge_pipeline.fit(X_train, y_train)
ridge_pred=ridge.predict(X_valid)
print("===ridge====")
print("mse==>",mean_squared_error(y_valid,ridge_pred))
print("mae==>",mean_absolute_error(y_valid,ridge_pred))
print("R^2==>",lasso.score(X_valid, y_valid))

===ridge====
mse==> 1321488665.4952872
mae==> 18693.778696102094
R^2==> 0.7115488730461104


In [ ]:
print("No: of Non zero coeficents",len([col for col in  ridge_pipeline.named_steps['ridge'].coef_ if abs(col) > 0 ]))

No: of Non zero coeficents 281


In [ ]:
elasticnet=elastic_pipeline.fit(X_train, y_train)
elasticnet_pred=elasticnet.predict(X_valid)
print("===elasticnet====")
print("mse==>",mean_squared_error(y_valid,elasticnet_pred))
print("mae==>",mean_absolute_error(y_valid,elasticnet_pred))
print("R^2==>",elasticnet.score(X_valid,y_valid))

===elasticnet====
mse==> 1709489293.2334154
mae==> 23623.86427839161
R^2==> 0.7591748794293667


In [ ]:
print("No: of Non zero coeficents",len([col for col in  elastic_pipeline.named_steps['elastic'].coef_ if abs(col) > 0 ]))

No: of Non zero coeficents 281


### The above results have demonstrated the fact that
###  1)Lasso helps in feature reduction by penalizing unwanted features to zero
###  2)Lasso regression helps in addressing  correlated features by shrinking one of the features selcted between the two
### features to zero
### 3)To get the best of Ridge and Lasso, elastic net addresses this by combining the advantages of lasso and ridge


In [ ]:
from numpy import array
from scipy.linalg import svd
# define a matrix
A = array([[4,11,14],[8,7,-2]])
print(A)
# SVD
U, s, VT = svd(A)
print(U)
print(s)
print(VT)

[[ 4 11 14]
 [ 8  7 -2]]
[[-0.9486833  -0.31622777]
 [-0.31622777  0.9486833 ]]
[18.97366596  9.48683298]
[[-0.33333333 -0.66666667 -0.66666667]
 [ 0.66666667  0.33333333 -0.66666667]
 [-0.66666667  0.66666667 -0.33333333]]


In [ ]:
A

array([[ 4, 11, 14],
       [ 8,  7, -2]])